# ♻️ EcoScan Self-Supervised Learning Enhancement
This notebook extends the EcoScan system with a self-supervised learning loop that leverages low-confidence predictions for self-improvement.

In [ ]:
# STEP 1: Install Required Libraries
!pip install transformers torch torchvision pillow

In [ ]:
# STEP 2: Load Model and Processor
from transformers import AutoModelForImageClassification, AutoImageProcessor
from PIL import Image
import torch
import os

model = AutoModelForImageClassification.from_pretrained("prithivMLmods/Recycling-Net-11")
processor = AutoImageProcessor.from_pretrained("prithivMLmods/Recycling-Net-11")
labels = model.config.id2label
model.eval()

In [ ]:
# STEP 3: Define a Self-Supervised Prediction Function
def predict_with_confidence(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
        confidence, predicted_class = torch.max(probs, dim=-1)
        return predicted_class.item(), confidence.item(), probs.squeeze().tolist()

In [ ]:
# STEP 4: Create a Pseudo-Labeled Dataset Based on Confidence
pseudo_labeled_data = []
image_folder = "./unlabeled_images"  # Replace with your folder
threshold = 0.75

for fname in os.listdir(image_folder):
    if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(image_folder, fname)
        pred_label, confidence, _ = predict_with_confidence(img_path)
        if confidence > threshold:
            pseudo_labeled_data.append((img_path, pred_label, confidence))
print(f"Collected {len(pseudo_labeled_data)} pseudo-labeled images above {threshold*100}% confidence.")

In [ ]:
# STEP 5: Fine-tune the Model Using Pseudo-Labeled Data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class PseudoLabelDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label, _ = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

pseudo_dataset = PseudoLabelDataset(pseudo_labeled_data, transform=transform)
pseudo_loader = DataLoader(pseudo_dataset, batch_size=16, shuffle=True)

In [ ]:
# STEP 6: Self-Supervised Fine-Tuning
from torch import nn, optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
model.train()

for epoch in range(3):
    total_loss = 0
    for images, labels in pseudo_loader:
        images = images.to(device)
        labels = labels.to(device)
        inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
        outputs = model(pixel_values=inputs['pixel_values'])
        loss = criterion(outputs.logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss/len(pseudo_loader):.4f}")